In [63]:
!pip install sentencepiece

  Using cached sentencepiece-0.2.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.7 kB/s eta 0:00:0000:0100:01


In [57]:
!pip install -U langsmith

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.1/149.1 kB 22.4 kB/s eta 0:00:00a 0:00:01


In [1]:
import os
import PyPDF2
from transformers import BertTokenizer, BertModel
from transformers import LongformerModel, LongformerTokenizer
from transformers import BigBirdModel, BigBirdTokenizer
import numpy as np
from groq import Groq
import gradio as gr
from pathlib import Path
import torch
import json


/home/sukhera/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from langsmith import Client

# Initialize the LangSmith Client
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "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"
client = Client()

In [3]:
# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

/home/sukhera/miniconda3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `b

In [4]:
# Load the BigBird model and tokenizer
tokenizer = BigBirdTokenizer.from_pretrained('google/bigbird-roberta-base')
model = BigBirdModel.from_pretrained('google/bigbird-roberta-base')


In [5]:
#longformer
# Load the Longformer model and tokenizer
tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')
model = LongformerModel.from_pretrained('allenai/longformer-base-4096')


In [28]:
#longFormer

def get_longformer_embedding(text):
    # Tokenize the text
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=4096)
    
    # Get the embeddings from Longformer
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Use the [CLS] token's embedding as the aggregate representation
    cls_embedding = outputs.last_hidden_state[:, 0, :].numpy()
    
    return cls_embedding

In [7]:
# BIGBIRD
def get_bigbird_embedding(text):
    # Tokenize the text
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=4096)
    
    # Get the embeddings from BigBird
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Use the [CLS] token's embedding as the aggregate representation
    cls_embedding = outputs.last_hidden_state[:, 0, :].numpy()
    
    return cls_embedding

In [8]:
def get_bert_embedding(text):
    # Tokenize the text
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    
    # Get the embeddings from BERT
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Use the [CLS] token's embedding as the aggregate representation
    cls_embedding = outputs.last_hidden_state[:, 0, :].numpy()
    
    return cls_embedding


In [9]:
def process_folder(file):
    folder_path = os.path.dirname(file.name)  # Get the directory of the selected file
    files = os.listdir(folder_path)  # List all files in the directory
    file_paths = [os.path.join(folder_path, f) for f in files]  # Get full paths of all files
    return f"Files in folder: {', '.join(files)}"

In [10]:
# Function to extract text from a PDF
def extract_text_from_pdf(pdf_file):
    text = ''
    with open(pdf_file, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            text += page.extract_text() or ''
    return text


In [11]:
cluster_emb={}

In [12]:

def calculate_cosine(embedding1, embedding2):
    # Calculate the dot product and magnitudes of the embeddings
    dot_product = np.dot(embedding1, embedding2)
    magnitude1 = np.linalg.norm(embedding1)
    magnitude2 = np.linalg.norm(embedding2)
    
    # Calculate cosine similarity
    similarity = dot_product / (magnitude1 * magnitude2)
    return similarity

In [24]:
def foo(files, JD):
    # Extract text and compute embeddings for job description using different models
    text_jd = extract_text_from_pdf(JD)    
    JD_embedding_bert = get_bert_embedding(text_jd).flatten()  # Flatten to match the dimension
    JD_embedding_longformer = get_longformer_embedding(text_jd).flatten()
    JD_embedding_bigbird = get_bigbird_embedding(text_jd).flatten()

    sim = []
    
    for d in files:
        text = extract_text_from_pdf(d)
        # Compute embeddings for the resume using different models
        resume_embedding_bert = get_bert_embedding(text).flatten()  # Fixed function call
        resume_embedding_longformer = get_longformer_embedding(text).flatten()
        resume_embedding_bigbird = get_bigbird_embedding(text).flatten()
        # Calculate cosine similarity for each model
        similarity_bert = calculate_cosine(resume_embedding_bert, JD_embedding_bert)
        similarity_longformer = calculate_cosine(resume_embedding_longformer, JD_embedding_longformer)
        similarity_bigbird = calculate_cosine(resume_embedding_bigbird, JD_embedding_bigbird)
        # Append the results to the array
        sim.append(f"\nFile: {d.name:}\n"
                   f"Bert Similarity: {similarity_bert:.4f}\n"
                   f"Longformer Similarity: {similarity_longformer:.4f}\n"
                   f"BigBird Similarity: {similarity_bigbird:.4f}\n")
        
    
    
    return "\n".join(sim)  # Join the list into a single string for Gradio output


In [25]:

with gr.Blocks() as func:
    inputs = [gr.File(file_count="multiple", label="Upload Resume Files"), gr.File(label="Upload Job Description")]
    outputs = gr.Textbox(label="Similarity Scores")
    show = gr.Button(value="Calculate Similarity")
    show.click(foo, inputs, outputs)

/home/sukhera/miniconda3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:2888: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/home/sukhera/miniconda3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:2888: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


In [29]:
func.launch()

Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----

To create a public link, set `share=True` in `launch()`.
